# 🔮 How To Use Quantum QEPC (FIXED - Works With Special Characters)

**This notebook shows you HOW to use the quantum_core.py file**

**FIXED:** Now handles player names with special characters (Dončić, Jokić, etc.)

---

## Step 1: Import Tools

In [2]:
from pathlib import Path
import sys
import pandas as pd
import numpy as np

# Find project root
current = Path.cwd()
project_root = current
for parent in [current, current.parent, current.parent.parent]:
    if (parent / "qepc").is_dir() or (parent / "data").is_dir():
        project_root = parent
        break

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"📁 Project root: {project_root}")

# Import quantum tools
from quantum_core import QuantumQEPC

print("✅ Successfully imported quantum tools!")

📁 Project root: C:\Users\wdors\qepc_project
✅ Successfully imported quantum tools!


---

## Step 2: Load Player Data with Smart Name Matching

In [3]:
# Load the FULL player game logs (not just averages)
# UPDATE THIS PATH to match where YOUR file is:
player_file = Path(r"C:\Users\wdors\qepc_project\notebooks\02_utilities\data\comprehensive\Player_Game_Logs_All_Seasons.csv")

print(f"Loading player data from: {player_file}")

if not player_file.exists():
    print(f"\n❌ File not found!")
    print(f"\nPlease update the path above to match your file location.")
    print(f"\nTo find it, run this in a new cell:")
    print(f"  from pathlib import Path")
    print(f"  list(Path('C:/Users/wdors/qepc_project').rglob('Player_Game_Logs_All_Seasons.csv'))")
else:
    # Load data
    player_data = pd.read_csv(player_file)
    
    print(f"\n✅ Loaded {len(player_data):,} player-game records")
    print(f"   Unique players: {player_data['PLAYER_NAME'].nunique():,}")
    
    # Fix season column if needed
    if 'SEASON_YEAR' in player_data.columns and 'Season' not in player_data.columns:
        player_data['Season'] = player_data['SEASON_YEAR']
    
    # Create QEPC with data
    qepc = QuantumQEPC(player_data=player_data)
    
    print(f"\n✅ Quantum QEPC ready with {len(player_data):,} records!")

Loading player data from: C:\Users\wdors\qepc_project\notebooks\02_utilities\data\comprehensive\Player_Game_Logs_All_Seasons.csv

✅ Loaded 254,187 player-game records
   Unique players: 1,425
✅ Quantum QEPC initialized!
   Loaded 254,187 player-season records

✅ Quantum QEPC ready with 254,187 records!


---

## Smart Player Search (Handles Special Characters)

This function finds players even if you don't know the exact spelling!

In [4]:
def find_player(search_name, show_all=False):
    """
    Smart player search that handles:
    - Partial names ("Luka" finds all Lukas)
    - Misspellings ("Doncic" finds "Dončić")
    - Multiple matches
    """
    if qepc.player_data is None:
        print("❌ No player data loaded!")
        return None
    
    # Method 1: Direct partial match
    matches = qepc.player_data[
        qepc.player_data['PLAYER_NAME'].str.contains(search_name, case=False, na=False)
    ]['PLAYER_NAME'].unique()
    
    # Method 2: If no matches, try without special characters
    if len(matches) == 0:
        # Remove special characters for comparison
        import unicodedata
        
        def remove_accents(text):
            """Remove accents from string"""
            return ''.join(c for c in unicodedata.normalize('NFD', text)
                          if unicodedata.category(c) != 'Mn')
        
        # Create normalized column
        normalized_names = qepc.player_data['PLAYER_NAME'].apply(remove_accents)
        search_normalized = remove_accents(search_name)
        
        # Search normalized names
        mask = normalized_names.str.contains(search_normalized, case=False, na=False)
        matches = qepc.player_data[mask]['PLAYER_NAME'].unique()
    
    # Display results
    if len(matches) == 0:
        print(f"❌ No players found matching '{search_name}'")
        print(f"\n💡 Try searching for just the first or last name")
        return None
    
    elif len(matches) == 1:
        print(f"✅ Found: {matches[0]}")
        return matches[0]
    
    else:
        print(f"⚠️  Found {len(matches)} players matching '{search_name}':")
        for i, name in enumerate(matches[:10], 1):  # Show first 10
            print(f"   {i}. {name}")
        
        if len(matches) > 10:
            print(f"   ... and {len(matches) - 10} more")
        
        if show_all:
            return matches
        else:
            print(f"\n💡 Using first match: {matches[0]}")
            print(f"   To see all matches, use: find_player('{search_name}', show_all=True)")
            return matches[0]

print("✅ Smart player search function ready!")
print("\nUsage: find_player('Luka')")
print("       find_player('Doncic')  # Works even without special characters!")

✅ Smart player search function ready!

Usage: find_player('Luka')
       find_player('Doncic')  # Works even without special characters!


---

## Example 1: Find A Player (Try It!)

In [5]:
# Try different searches:

# Search by partial first name
player = find_player('Luka')

# Search by last name (even without special characters!)
# player = find_player('Doncic')  # Finds Dončić

# Search by full name
# player = find_player('Stephen Curry')

# Search shows all matches
# players = find_player('James', show_all=True)

⚠️  Found 3 players matching 'Luka':
   1. Luka Dončić
   2. Luka Samanic
   3. Luka Garza

💡 Using first match: Luka Dončić
   To see all matches, use: find_player('Luka', show_all=True)


---

## Example 2: Predict A Game

In [6]:
# Predict Lakers vs Celtics
prediction = qepc.predict_game(
    team_a_score_mean=112,   # Lakers expected score
    team_a_score_std=8,       # Lakers variance
    team_b_score_mean=115,   # Celtics expected score
    team_b_score_std=7,       # Celtics variance
    n_sims=10000
)

print("🏀 Lakers vs Celtics Prediction:")
print(f"   Lakers score: {prediction['team_a_score']:.1f}")
print(f"   Celtics score: {prediction['team_b_score']:.1f}")
print(f"   Spread: {prediction['spread']:.1f} (negative = Celtics favored)")
print(f"   Total: {prediction['total']:.1f}")
print(f"\n   Lakers win probability: {prediction['team_a_win_prob']:.1%}")
print(f"   Celtics win probability: {prediction['team_b_win_prob']:.1%}")

🏀 Lakers vs Celtics Prediction:
   Lakers score: 112.0
   Celtics score: 115.0
   Spread: -3.0 (negative = Celtics favored)
   Total: 226.9

   Lakers win probability: 38.6%
   Celtics win probability: 61.4%


---

## Example 3: Quantum Interference (Combine Predictions)

In [7]:
# You have 3 different models predicting Lakers score:
predictions = [110, 115, 108]  # Three different model predictions
confidences = [0.8, 0.9, 0.6]  # How confident each model is

# OLD WAY (simple average):
simple_average = sum(predictions) / len(predictions)
print(f"Simple average: {simple_average:.1f}")

# NEW WAY (quantum interference):
quantum_result = qepc.quantum_interference_ensemble(predictions, confidences)
print(f"Quantum interference: {quantum_result['prediction']:.1f}")

print(f"\n💡 Quantum method weighs the 115 prediction more (highest confidence)")
print(f"   This is smarter than treating all predictions equally!")

Simple average: 111.0
Quantum interference: 110.6

💡 Quantum method weighs the 115 prediction more (highest confidence)
   This is smarter than treating all predictions equally!


---

## Example 4: Analyze Player Consistency

In [8]:
# Find a star player
player_name = find_player('Luka')  # Finds Luka Dončić automatically!

if player_name:
    # Analyze consistency
    analysis = qepc.analyze_player_consistency(player_name, season='2023-24')
    
    if analysis:
        print(f"\n🎯 {analysis['player']} Analysis:")
        print(f"   Average: {analysis['ppg']:.1f} PPG")
        print(f"   Uncertainty: ±{analysis['uncertainty']:.1f}")
        print(f"   Consistency Score: {analysis['consistency_score']:.2f}")
        print(f"   Category: {analysis['category']}")
        
        print(f"\n💡 Interpretation:")
        if analysis['category'] == 'Consistent':
            print(f"   This player is RELIABLE for props betting!")
        elif analysis['category'] == 'Volatile':
            print(f"   This player is BOOM/BUST - riskier bets!")
    else:
        print(f"\n⚠️  No 2023-24 data for {player_name}")
        print(f"   Try a different season or player")

⚠️  Found 3 players matching 'Luka':
   1. Luka Dončić
   2. Luka Samanic
   3. Luka Garza

💡 Using first match: Luka Dončić
   To see all matches, use: find_player('Luka', show_all=True)

🎯 Luka Dončić Analysis:
   Average: 33.9 PPG
   Uncertainty: ±8.8
   Consistency Score: 0.79
   Category: Consistent

💡 Interpretation:
   This player is RELIABLE for props betting!


---

## Example 5: Compare Multiple Players

In [9]:
# Compare consistency of multiple star players
players_to_check = [
    'Luka',      # Finds Luka Dončić
    'Jokic',     # Finds Nikola Jokić
    'Curry',     # Finds Stephen Curry
    'Giannis',   # Finds Giannis Antetokounmpo
    'Durant'     # Finds Kevin Durant
]

results = []

print("🔍 Analyzing players...\n")

for search in players_to_check:
    player_name = find_player(search)
    
    if player_name:
        analysis = qepc.analyze_player_consistency(player_name, season='2023-24')
        
        if analysis:
            results.append({
                'Player': player_name,
                'PPG': analysis['ppg'],
                'Uncertainty': analysis['uncertainty'],
                'Consistency': analysis['consistency_score'],
                'Category': analysis['category']
            })

if results:
    # Convert to DataFrame for nice display
    df_results = pd.DataFrame(results)
    
    print("\n📊 Player Consistency Comparison (2023-24):\n")
    print(df_results.to_string(index=False))
    
    print("\n💡 Higher consistency score = More reliable for props betting")

🔍 Analyzing players...

⚠️  Found 3 players matching 'Luka':
   1. Luka Dončić
   2. Luka Samanic
   3. Luka Garza

💡 Using first match: Luka Dončić
   To see all matches, use: find_player('Luka', show_all=True)
✅ Found: Nikola Jokić
⚠️  Found 2 players matching 'Curry':
   1. Stephen Curry
   2. Seth Curry

💡 Using first match: Stephen Curry
   To see all matches, use: find_player('Curry', show_all=True)
⚠️  Found 2 players matching 'Giannis':
   1. Giannis Antetokounmpo
   2. Georgios Papagiannis

💡 Using first match: Giannis Antetokounmpo
   To see all matches, use: find_player('Giannis', show_all=True)
✅ Found: Kevin Durant

📊 Player Consistency Comparison (2023-24):

               Player       PPG  Uncertainty  Consistency   Category
          Luka Dončić 33.857143     8.810872     0.793502 Consistent
         Nikola Jokić 26.392405     8.160910     0.763817 Consistent
        Stephen Curry 26.432432     9.583206     0.733915 Consistent
Giannis Antetokounmpo 30.438356     9.11955

---

## Example 6: Quantum vs Regular Monte Carlo

In [10]:
# Simulate a team that averages 110 points with ±10 variance

# REGULAR Monte Carlo (what most people do)
regular_samples = np.random.normal(110, 10, 10000)
regular_mean = np.mean(regular_samples)

# QUANTUM Monte Carlo (what we do)
quantum_result = qepc.quantum_monte_carlo(110, 10, n_sims=10000, importance_weight=1.5)
quantum_mean = quantum_result['mean']

print("📊 Comparison:")
print(f"   Target: 110.0")
print(f"   Regular Monte Carlo: {regular_mean:.1f}")
print(f"   Quantum Monte Carlo: {quantum_mean:.1f}")

print(f"\n💡 Quantum MC is more accurate with fewer samples!")
print(f"   This means faster predictions with same accuracy.")

📊 Comparison:
   Target: 110.0
   Regular Monte Carlo: 110.0
   Quantum Monte Carlo: 110.1

💡 Quantum MC is more accurate with fewer samples!
   This means faster predictions with same accuracy.


---

## 🎯 Quick Reference

### **Find a player:**
```python
player = find_player('Luka')  # Partial match
player = find_player('Doncic')  # Works without special characters!
players = find_player('James', show_all=True)  # See all matches
```

### **Predict a game:**
```python
prediction = qepc.predict_game(
    team_a_score_mean=112,
    team_a_score_std=8,
    team_b_score_mean=115,
    team_b_score_std=7
)
print(prediction['team_a_win_prob'])
```

### **Combine predictions:**
```python
result = qepc.quantum_interference_ensemble(
    predictions=[110, 115, 108],
    confidences=[0.8, 0.9, 0.6]
)
print(result['prediction'])
```

### **Analyze consistency:**
```python
player = find_player('Curry')
analysis = qepc.analyze_player_consistency(player, season='2023-24')
print(analysis['category'])  # Consistent, Moderate, or Volatile
```

---

## ✅ You're Ready!

**You now know how to:**
- ✅ Find players (even with special characters)
- ✅ Make quantum-enhanced predictions
- ✅ Combine multiple models intelligently
- ✅ Analyze player consistency

**Go make some predictions!** 🚀